In [2]:
import json

In [5]:
# Load file with training configuration
with open('training_config.json') as f:
    train_config = json.load(f)
  

{'data': {'subsample': 0, 'path_data': './data', 'path_data_train': './data/dataset_train.pkl', 'path_data_val': './data/dataset_val.pkl', 'path_data_test': './data/dataset_test.pkl', 'path_results': './results', 'path_eda': './eda', 'random_state': 42}, 'skip': ['Linear_onlyT', 'Poly_2', 'Arrh_onlyT', 'Linear_onlyT', 'Linear', 'Poly_2', 'Poly_3', 'Arrh_onlyT', 'Linear_Arrh', 'Poly2_Arrh', 'Autofeat_1', 'Autofeat_2', 'Autofeat_3', 'Autofeat_4', 'Autofeat_5', 'Autofeat_6', 'Selected_2'], 'models': {'Linear_onlyT': {'scaling': 'standard', 'stdalpha': 1, 'thresshold': None, 'features': ['T'], 'workflow': 'NoGen', 'fit_intercept': True}, 'Linear': {'scaling': 'standard', 'stdalpha': 1, 'thresshold': None, 'features': ['T', 'Li', 'PC'], 'workflow': 'NoGen', 'fit_intercept': True}, 'Poly_2': {'scaling': 'standard', 'stdalpha': 1, 'thresshold': 2, 'features': ['T', 'Li', 'PC'], 'poly_order': 2, 'workflow': 'Poly', 'fit_intercept': True}, 'Poly_3': {'scaling': 'standard', 'stdalpha': 1, 'thres

In [ ]:
def load_data():
    """
    Function to load data. In this example data is artificially generated from a function.
    However in real use scenarios you might replace this function by another loading your
    dataset from disk (e.g. loading a pickle, or a csv)"""
    pass